In [ ]:
# Importar e instalar dependencias
!pip install sacremoses
import pandas as pd
import numpy as np
import re
from collections import Counter

# CARGA DE DATOS

In [ ]:
!pip install gdown
# Descarga el archivo del dataset de drive usando gdown
url = 'https://drive.google.com/file/d/1LkEJ3rstkdyhUGWi9O2YKQMe0wC_ZyDd/view?usp=sharing'
file_id = url.split('/')[-2]
!gdown --id $file_id

# Carga el dataset usando pandas
sd = pd.read_csv('/content/Suicide_Detection.csv')

In [ ]:
from transformers import pipeline, MarianTokenizer, MarianMTModel
import pandas as pd
from tqdm import tqdm
import torch
!pip install sacremoses

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Sample data for testing purposes
sd_t = sd.sample(n=200, random_state=None)

# Load a specific tokenizer and model
tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es", clean_up_tokenization_spaces=True)
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-es")

# Create a translation pipeline with the specified tokenizer and model
translator = pipeline("translation_en_to_es", model=model, tokenizer=tokenizer, device=device)

def translate_batch(texts, batch_size=32):  # Adjust batch_size for your GPU memory
    translations = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i + batch_size]
        # Tokenize the batch of texts
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to(device)  # Move inputs to the device
        # Generate translations
        translated_batch = model.generate(**inputs)
        # Decode translations
        decoded_batch = tokenizer.batch_decode(translated_batch, skip_special_tokens=True)
        translations.extend(decoded_batch)
    return translations

# Translate the texts in batches
sd_t['spanish_text'] = translate_batch(sd_t['text'].tolist())

# Verify by printing the first few rows
print(sd_t.head())



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Define la ruta donde quieres guardar el archivo en tu Google Drive
file_path = '/content/drive/MyDrive/Suicide_Detection_translated_5.csv'

# Guarda el DataFrame en un archivo CSV en Google Drive
sd_t.to_csv(file_path, index=False)

In [ ]:
sd_t.head(20)